In [1]:
import os 
os.chdir("../")

In [2]:
from src.utils.helpers import _getActivityNames, makeProgressBar, safe_update_bar

/home/vinh/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from pm4py.objects.log.obj import EventLog
from typing import Dict, List, Set
import pm4py.util.xes_constants as xes
from sklearn.cluster import DBSCAN
import numpy as np
from numpy.typing import NDArray

In [4]:
from pm4py import read_xes
log = read_xes('data/bose_log.xes')

parsing log, completed traces :: 100%|████| 6000/6000 [00:03<00:00, 1780.03it/s]


In [9]:
def calcRelationMatrix(log:EventLog, activityName_key:str=xes.DEFAULT_NAME_KEY, progress=None)->np.ndarray:
    """Calculate the Relation Matrix defined by Zheng et al. Contains for each Case, and each Eventually- and Directly-Follows relation a 1 if it holds in the case, and 0 otherwise.

    Args:
        log (EventLog): The event log.
        activityName_key (str, optional): The key for the activity value in the event log. Defaults to xes.DEFAULT_NAME_KEY.
        progress (Any, optional): A progress bar to update at every completed case. Defaults to None.

    Returns:
        np.ndarray: The calculated Relation Matrix. Contains for each Case, and each Eventually- and Directly-Follows relation a 1 if it holds in the case, and 0 otherwise. Dimensions: 2(num_activities^2) x len(log)
    """    

    activities = _getActivityNames(log, activityName_key=activityName_key)
    num_activities = len(activities)

    # Each column corresponds to a trace
    # Each Row corresponds to a Relation
    print(activities)
    drelmatrix = np.zeros( # Directly follows relations
        ((num_activities**2), len(log))
    ) # Initialize to 0, so we can just look at the 1's
    wrelmatrix = np.zeros( # Weak Relations
        ((num_activities**2), len(log))
    ) # Initialize to 0, so we can just look at the 1's
    for idx,trace in enumerate(log):
        # idx is the column
        # Set the cell to 1 where there is a directly (or weak) follows relation between the activities
        seen = set() # Set of indices of the activities we have already seen in the trace. Used to update weak relations
        for i in range(len(trace)-1): # Exclude last element as it has no follower
            act1_index = activities.index(trace[i][activityName_key])
            act2_index = activities.index(trace[i+1][activityName_key])
            # print(trace[i][activityName_key])
            # print(trace[i+1][activityName_key])
            # print("this is activity 1 index", act1_index)
            # print("this is activity 2 index", act2_index)
            # Update Directly follows Cell for this relation
            drelmatrix[act1_index*num_activities + act2_index, idx] = 1
            wrelmatrix[act1_index*num_activities + act2_index, idx] = 1
            for act in seen:
                wrelmatrix[act*num_activities + act1_index,idx]
            seen.add(act1_index)
        safe_update_bar(progress)
    return np.append(drelmatrix,wrelmatrix,axis=0)

In [6]:
# => it's like 
'''                trace1   trace2   trace3   trace4   trace5  
act1-> act1          
act1-> act2
....




'''

'                trace1   trace2   trace3   trace4   trace5  \nact1-> act1          \nact1-> act2\n....\n\n\n\n\n'

In [7]:
def candidateCPDetection(relMatrixRow:NDArray, mrid:int)->Set[int]:
    """Extract candidate change points from a row of the Relation Matrix. These are points where the relation of this row held (or did not hold) for `mrid` consecutive traces, and then the relation changed.

    Args:
        relMatrixRow (NDArray): A row of the Relation Matrix.
        mrid (int): The Minimum Relation Invariance Distance. How long a relationship must remain stable before its change is concidered a change point candidate.
        a row represent a relation, each column is a trace, that mean it is a array of value 0,0,0,0,1,0,0,1,0,0 ....
    Returns:
        Set[int]: A set of indices of the change point candidates.
    """    

    P = set()
    begin = 0
    count = 0
    n = len(relMatrixRow)
    print("the length is", n)
    for j in range(n): # n+1 so n is included
        if begin == 0 or relMatrixRow[j] != relMatrixRow[begin]: # begin = 0 mean th first step, relaM[j] != relaM[begin] mean the the realation in the step j diff with the relation in begin step 
            if count >= mrid: # at first step there is never true. if mrid > 0. 
                P.update([begin, j])
            begin = j # update begin to next step begin until relaM[j] != relaM[begin]
            count = 0 # count reset
        count += 1 # if relaM[j] == relaM[begin]
    if count >= mrid:
        P.add(begin)
    P.difference_update([1])
    return P

In [17]:
def candidateChangepointsCombinataion(S:Set[int], mrid:int, eps:float, n:int)->List[float]:
    """Combine candidate change points by clustering them (Using DBScan), to find change point indices

    Args:
        S (Set[int]): The set of found candidate change points (From all Matrix rows).
        mrid (int): The Minimum Relation Invariance Distance. How long a relationship must remain stable before its change is concidered a change point candidate.
        eps (float): The epsilon parameter used for the DBSCAN clustering algorithm.
        n (int): The index of the last trace in the log. Zheng et al. consider this, and the first case, change point candidates as well.

    Raises:
        Exception: No neighbor change points were found for a detected change point. This should not occur due to the fact that `1` and `n` are also considered change points.

    Returns:
        List[float]: A list of change point indices. These are floats due to the use of a clustering algorithm.
    """    


    minPts = 1
    result = [1,n]

    if S == set(): # If there are no candidates
        # Otherwise DBSCAN will yield an error
        return result

    s_as_list = list(S) # So i can be certain about the order
    clustering = DBSCAN(eps=eps, min_samples=minPts).fit(np.array(s_as_list).reshape(-1, 1))

    map_to_cluster = []
    for i in range(len(s_as_list)):
        map_to_cluster.append(
            (s_as_list[i], clustering.labels_[i])
        )
    cluster_names = set(clustering.labels_) # All the different clusters
    clusters = [
        (cluster_name, [
            pt for pt,clust in map_to_cluster if clust == cluster_name
        ]) for cluster_name in cluster_names
    ] # "Maps" each cluster name to the list of elements in it
    # Remove empty clusters (because apparently i have to do that??)
    clusters = [(x,y) for x,y in clusters if y != []]

    # Sort it
    clusters.sort(key=lambda x: len(x[1]), reverse=True)# Sort w.r.t. the size of the cluster, descending
    for clust_name, clust_members in clusters:
        c = np.average(clust_members)
        leftIndex = None
        rightIndex = None
        for i in range(len(result)-1):
            if result[i] < c and c < result[i+1]:
                leftIndex = i
                rightIndex = i+1
                break;
        if rightIndex is not None and leftIndex is not None:
            if c-leftIndex >= mrid and rightIndex-c<= mrid:
                result.insert(rightIndex,c)
        else:
            # This should not happen
            raise Exception("Couldn't find a place in results list for changepoint. This should not happen, contact the developer")
    return result


In [10]:
out = calcRelationMatrix(log)

['Archive', 'Contact Hospital', 'Create Questionnaire', 'High Insurance Check', 'High Medical History', 'Low Insurance Check', 'Low Medical History', 'Prepare Notification Content', 'Receive Questionnaire Response', 'Register', 'Send Notification by Phone', 'Send Notification by Post', 'Send Notification by e-mail', 'Send Questionnaire', 'Skip Questionnaire']


In [33]:
P = set()
for row in out:
    P.update(candidateCPDetection(row, mrid=600))


In [29]:
P

{51,
 136,
 223,
 224,
 288,
 289,
 323,
 324,
 339,
 486,
 575,
 579,
 580,
 651,
 757,
 803,
 856,
 857,
 869,
 978,
 982,
 983,
 995,
 999,
 1089,
 1137,
 1175,
 1187,
 1188,
 1190,
 1192,
 1200,
 1225,
 1226,
 1239,
 1254,
 1287,
 1288,
 1289,
 1290,
 1380,
 1381,
 1392,
 1410,
 1424,
 1425,
 1439,
 1440,
 1569,
 1594,
 1602,
 1657,
 1700,
 1745,
 1746,
 1748,
 1773,
 1798,
 1799,
 1861,
 1875,
 1882,
 1891,
 1895,
 1956,
 1990,
 1991,
 2051,
 2093,
 2117,
 2134,
 2152,
 2216,
 2217,
 2231,
 2345,
 2353,
 2393,
 2414,
 2418,
 2540,
 2558,
 2639,
 2698,
 2725,
 2752,
 2817,
 2868,
 2934,
 2967,
 2979,
 2999,
 3065,
 3066,
 3083,
 3084,
 3090,
 3119,
 3136,
 3156,
 3157,
 3268,
 3276,
 3278,
 3306,
 3307,
 3373,
 3394,
 3395,
 3454,
 3501,
 3503,
 3516,
 3573,
 3588,
 3593,
 3599,
 3600,
 3634,
 3646,
 3667,
 3707,
 3733,
 3799,
 3800,
 3804,
 3840,
 3903,
 3947,
 3968,
 4015,
 4016,
 4017,
 4031,
 4032,
 4037,
 4061,
 4077,
 4079,
 4110,
 4111,
 4147,
 4148,
 4200,
 4243,
 4244,
 42

In [34]:
cp = candidateChangepointsCombinataion(P, mrid=600, eps=300, n=len(log))

In [35]:
cp = [round(x) for x in cp]

In [36]:
cp

[1, 1278, 1971, 3731, 5199, 6000]

In [38]:
log[0]

{'attributes': {'concept:name': '1', 'variant': 'Variant 875', 'variant-index': 875, 'creator': 'Fluxicon Disco'}, 'events': [{'concept:name': 'Register', 'lifecycle:transition': 'complete', 'org:resource': 'PR3', 'time:timestamp': datetime.datetime(1970, 1, 1, 1, 6, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'Activity': 'Register', 'Resource': 'PR3'}, '..', {'concept:name': 'Archive', 'lifecycle:transition': 'complete', 'org:resource': 'PR1', 'time:timestamp': datetime.datetime(1970, 1, 2, 7, 39, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), 'Activity': 'Archive', 'Resource': 'PR1'}]}

In [39]:
def extractTraces(log: EventLog, activityName_key:str=xes.DEFAULT_NAME_KEY)-> np.ndarray:
    out = np.empty(len(log), dtype=object)
    for index, case in enumerate(log):
        out[index] = tuple(evt[activityName_key] for evt in case)
    return out

In [40]:
traces = extractTraces(log)
traces[0]

('Register',
 'Low Insurance Check',
 'Create Questionnaire',
 'Low Medical History',
 'Prepare Notification Content',
 'Send Questionnaire',
 'Send Notification by Post',
 'Send Notification by e-mail',
 'Receive Questionnaire Response',
 'Archive')

In [41]:
len(traces)

6000

In [43]:
for i in range(5):
    print(traces[i])

('Register', 'Low Insurance Check', 'Create Questionnaire', 'Low Medical History', 'Prepare Notification Content', 'Send Questionnaire', 'Send Notification by Post', 'Send Notification by e-mail', 'Receive Questionnaire Response', 'Archive')
('Register', 'Low Insurance Check', 'Low Medical History', 'Prepare Notification Content', 'Create Questionnaire', 'Send Notification by e-mail', 'Send Questionnaire', 'Skip Questionnaire', 'Archive')
('Register', 'Create Questionnaire', 'Low Medical History', 'Low Insurance Check', 'Prepare Notification Content', 'Send Questionnaire', 'Skip Questionnaire', 'Archive')
('Register', 'Create Questionnaire', 'High Insurance Check', 'High Medical History', 'Contact Hospital', 'Prepare Notification Content', 'Send Notification by Post', 'Send Notification by Phone', 'Send Questionnaire', 'Skip Questionnaire', 'Archive')
('Register', 'High Insurance Check', 'Contact Hospital', 'High Medical History', 'Create Questionnaire', 'Prepare Notification Content',